In [ ]:
# bands

In [1]:

from sentinelhub import BBox, CRS, DataCollection, SHConfig, WmsRequest, WcsRequest, DataSource, MimeType
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
import rasterio
import numpy as np
import datetime

import os 
from osgeo import gdal
import glob

import imageio

from PIL import Image
import io

from rasterio.transform import from_bounds

import rasterio
from matplotlib import pyplot
from rasterio.plot import show

import numpy as np
import os 

import imageio


INSTANCE_ID = '1f1f36b7-f080-4951-bc16-efb302fa42ec'
if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

print(config)


{
  "instance_id": "1f1f36b7-f080-4951-bc16-efb302fa42ec",
  "sh_client_id": "",
  "sh_client_secret": "",
  "sh_base_url": "https://services.sentinel-hub.com",
  "geopedia_wms_url": "https://service.geopedia.world",
  "geopedia_rest_url": "https://www.geopedia.world/rest",
  "aws_access_key_id": "",
  "aws_secret_access_key": "",
  "aws_metadata_url": "https://roda.sentinel-hub.com",
  "aws_s3_l1c_bucket": "sentinel-s2-l1c",
  "aws_s3_l2a_bucket": "sentinel-s2-l2a",
  "opensearch_url": "http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2",
  "max_wfs_records_per_query": 100,
  "max_opensearch_records_per_query": 500,
  "max_download_attempts": 4,
  "download_sleep_time": 5,
  "download_timeout_seconds": 120,
  "number_of_download_processes": 1
}


In [3]:
# BANDS TO DOWNLOAD:

# Green, red, SWIR, NIR, NDSI, NDFSI



class SentinelData:
    """
    Retreiveing Sentinel data from Sentinel Hub
    initialise class with coordinates list, resolution, bounding box coordinates, years and month ranges
    make a wcs request for data between November-April
    """
    def __init__(self, coords_list, years, day_month_to, day_month_from):
        self.coords_list = coords_list
       # self.resolution = '10m'
        self.area_coords = BBox(bbox=self.coords_list, crs=CRS.WGS84)
        self.year_range = years # make a set of years to iterate over 
        self.day_month_to = day_month_to
        self.day_month_from = day_month_from
        
# E.g. setting resx='10m' and resy='10m' will return an image where every pixel will cover an area of size 10m x 10m.        
        
    def retrieve_data(self):
        """ 
        Create a bounding box and assign CRS. 
        Create a wcs data request from Sentinel for Sentinel-2 data. 
        Define max clouds as 20%
        """
        for year in self.year_range:
            time_from = "{}-{}".format(year, self.day_month_from)
           # print("time_from", time_from)
            time_to = "{}-{}".format(year + 1,  self.day_month_to)
           # print("time_to", time_to)
            # make the request for the desired date range 
            wcs_true_color_request = WcsRequest (
                data_collection=DataCollection.SENTINEL2_L1C,
                layer='TRUE-COLOR-S2-L2A', # Layer you have configured
                bbox=self.area_coords,
                time= (time_from, time_to),
                resx= '10m', #self.resolution,  
                resy= '10m', #self.resolution, 
                config=config,
                maxcc=0 
            )
    
            available_dates_list = wcs_true_color_request.get_dates()
            yield from available_dates_list
            
    
    def get_available_data(self):
        """
        Use get_data() function from Sentinel to retrieve available data for the dates.
        """
        available_dates_list = self.retrieve_data()
        i = 0
        for date in available_dates_list:
            i += 1
            print("I", i)
        
            wcs_true_color_request = WcsRequest(
                data_collection=DataCollection.SENTINEL2_L1C,
                layer='TRUE-COLOR-S2-L2A', # Layer you have configured
                bbox=self.area_coords, 
                time= date,
                resx= '10m', #self.resolution, # Stick to 10m resolution as this the maximum possible 
                resy= '10m', #self.resolution, 
                config=config,
                image_format=MimeType.TIFF,
                maxcc=0,# You can define the maximum ammount of cloud coverage you want to allow.
                data_folder='/Users/emilybirch/Documents/UCL_Dissertation/Resampling/all_bands' 
            )
            
            basemap = wcs_true_color_request.get_data()[0] 
            
            # call the bands_req function to wms request for bands on available dates
            call_wms = self.bands_req(basemap, date, i) # basemap,
            
           # print('Returned data is of type = %s and length %d.' % (type(basemap), len(basemap)))
          #  print(f'Single element in the list is of type {type(basemap[-1])} and has shape {basemap[-1].shape}')
           
            
            

            
    def bands_req(self, basemap, date, i):
        """
        wms bands request for the bands making up the Sentinel pic
        """
        basemap = np.array(basemap)
        wms_bands_request = WmsRequest(
            data_collection=DataCollection.SENTINEL2_L1C,
            layer='BANDS-S2-L2A', 
            bbox= self.area_coords, 
            width=basemap.shape[1], # 10m resolution dims are sourced from the basemap. 
            height=basemap.shape[0],
            time=(date),
            image_format=MimeType.TIFF, 
            maxcc=0,
            config=config,
            data_folder='/Users/emilybirch/Documents/UCL_Dissertation/Resampling/all_bands' 
        )        
        
        bands = wms_bands_request.get_data()[-1] # save_data=True
        bands = np.array(bands)
        bands = bands.astype('float32') 
           
        # call the split_bands function
        get_channels = self.split_bands(bands, basemap, i)

 


    def split_bands(self, bands, basemap, i):
        """
        Red b04
        NIR b08
        SWIR b11
        NDSI (b03 - b11)/(b03 + b11)
        NDFSI (NIR-SWIR)/(NIR +SWIR)
        
        
        
        looking at pics maybe these:
        
        red_ndfsi_green !!! DO THIS COMBO
        or basic- nir_swir_red
        or NSDI_NDFSI_red
        
        ** ndsi misclassifies lakes/water so badly, do not use!
        
       """
        
        b01 = bands[:,:,0] # Coastal Aerosol
        b02 = bands[:,:,1] # Blue
        b03 = bands[:,:,2] # Green
        b04 = bands[:,:,3] # Red
        b05 = bands[:,:,4] # Vegetation Red Edge 
        b06 = bands[:,:,5] # Vegetation Red Edge
        b07 = bands[:,:,6] # Vegetation Red Edge
        b08 = bands[:,:,7] # NIR
        b08a = bands[:,:,8] # Vegetation Red Edge
        b09 = bands[:,:,9] # Water Vapour
        b11 = bands[:,:,10] # SWIR
        b12 = bands[:,:,11] # SWIR
        
        
        out_path = ('CN_bbox2_{}.tif'.format(i))
        print("outpath", out_path)
        
        if i == 82 or i == 83 or i ==87: 
            # Red band first b04
            os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Red_b04')
            imageio.imwrite(out_path, b04) #.astype('uint8'))

            # test green band first b03
            os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Green_b03')
            imageio.imwrite(out_path, b03) #.astype('uint8')) .astype('float32')

            # NIR b08
            os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/NIR_b08')
            imageio.imwrite(out_path, b08) 

            # SWIR b11
            os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/SWIR_b11')
            imageio.imwrite(out_path, b11) 


            # NDSI
            NDSI = (b03 - b11)/(b03 + b11)
            os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/NDSI')
            imageio.imwrite(out_path, NDSI)


            # NDFSI (NIR-SWIR)/(NIR +SWIR)
            NDFSI = (b08 - b11)/(b08 + b11)
            os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/NDFSI')
            imageio.imwrite(out_path, NDFSI) 
            
        else:
            print("not writing file:", out_path)


            
            

# when if statement works do big ones:
# cn2, fr3
# DLETE UNUSED CANADA BOX 2!!!





            
# what are the pic numbers for if statement to save/ to keep in folder:
# can 1: 1,2,4,5 # they were 1,2,3,4,5? switch around, check pics
# can 2: 82,83,87 
# can 3: 1,2,3,4,5 


# fr 1: 2,5,7,8 
# fr 2: 2,4,6
# fr 3: 15,17,18,25 

# nz 1: 6,8,10 
# nz 2: 3,4,5 
# nz 3: 2,3,6
# nz 4: 3,13

# nor1: 14,15,18,22,36 
# nor 2: 1,3,5,12
# nor 3: 3,7,14,15

# arg 1: 1,2,5,7,8 
# arg 2: 1,2,7
# arg 3: 2,3,4,5


# al 1: 2,3,4 
# al 2: 2,3, 
# al 3: 2,3,4
# al 4: 2,4,6









# get rid can sat box 2- 83 is too far down


# how many pics per box 
 # - canada: box1 x 4 , b2 x 3, b3 x 5
# - France: box1 x 4, b2 x 4, b3 x 4
# - NZ: box1 x 3, b2 x 3, b3 x 3, b4 x 3
# - Alaska: box1 x 4, b2 x 2, b3 x 3, b4 x 3
# - Argentina: b1 x 5, b2 x 3, b3 x 4 
#- Norway: b1 x5, b2 x 4, b3 x 4       
        
    
    
# 1. Canada bands- 

# define the years and month ranges for data. Nov-April so there is snow cover.
# Sentinel 2 data is only available from 2015 onwards
canada_years = {2014, 2015, 2016, 2017, 2018, 2019, 2020}

# define the years and month ranges for data. Nov-April
canada_day_month_to = "01-01"
canada_day_month_from = "01-01"

# canada 0%CC. BBOX1. CANADIAN ROCKIES
s4, w4 = 50.82675, -116.08768
n4, e4 = 51.02583, -115.73474       


# CANADA BBOX2 0%CC. VANCOUVER. takes a long time to plot (90 plots)
s5, w5 = 49.60259, -123.27264
n5, e5 = 49.80043, -122.94511
    

# canada BBOX3. nr vancouver 
s6, w6 = 49.98434, -124.25091
n6, e6 = 50.19579, -123.90758
       
    
coords_canada1 = [w4, s4, e4, n4]
coords_canada2 = [w5, s5, e5, n5]
coords_canada3 = [w6, s6, e6, n6]

can_data1 = SentinelData(coords_canada1, canada_years, canada_day_month_to, canada_day_month_from)
can_data2 = SentinelData(coords_canada2, canada_years, canada_day_month_to, canada_day_month_from)
can_data3 = SentinelData(coords_canada3, canada_years, canada_day_month_to, canada_day_month_from)

# wcs 
#plot_data_1 = canada_data.plot_data()


# get wms
#plot_canada_wms1 = can_data1.get_available_data()
plot_canada_wms2 = can_data2.get_available_data() # BOX 2 HAS WAY TOO MANY PICS. LEAVE
#plot_canada_wms3 = can_data3.get_available_data() 
    
    

# FRANCE BBOX 1. 0%CC
s, w = 46.00263, 6.68625
n, e = 46.21018, 7.00556


# FRANCE BBOX 2. 0%CC
s2, w2 = 44.5032, 6.26995
n2, e2 = 44.71395, 6.57683

# FRANCE BBOX 3. 0%CC. in Vosges mountains. 
s3, w3 = 47.89925, 6.75874
n3, e3 = 48.0968, 7.0709
    
coords_fr1 = [w, s, e, n]
coords_fr2 = [w2, s2, e2, n2]
coords_fr3 = [w3, s3, e3, n3]

fr_data1 = SentinelData(coords_fr1, canada_years, canada_day_month_to, canada_day_month_from)
fr_data2 = SentinelData(coords_fr2, canada_years, canada_day_month_to, canada_day_month_from)
fr_data3 = SentinelData(coords_fr3, canada_years, canada_day_month_to, canada_day_month_from)


# get wms
#plot_fr_wms1 = fr_data1.get_available_data()
#plot_fr_wms2 = fr_data2.get_available_data()
#plot_fr_wms3 = fr_data3.get_available_data()


# NEW ZEALAND 0% CC. BBOX1. FORESTED. gets 3 3/4 pics 
s7, w7 = -44.64093, 168.67527
n7,e7 = -44.42578, 168.99039


# NZ BBOX 2 FORESTED. 3 pics  
s8,w8 = -44.46215, 169.15172
n8,e8 = -44.26874, 169.4559


# NZ BBOX 3. 3 pics 
s9,w9 = -45.57742, 167.32814
n9,e9 = -45.37906, 167.63988


# NZ bbox 4. 
s10,w10 = -44.21029, 168.7185
n10, e10 = -43.99234, 169.00415
    


coords_nz1 = [w7, s7, e7, n7]
coords_nz2 = [w8, s8, e8, n8]
coords_nz3 = [w9, s9, e9, n9]
coords_nz4 = [w10, s10, e10, n10]


nz_data1 = SentinelData(coords_nz1, canada_years, canada_day_month_to, canada_day_month_from)
nz_data2 = SentinelData(coords_nz2, canada_years, canada_day_month_to, canada_day_month_from)
nz_data3 = SentinelData(coords_nz3, canada_years, canada_day_month_to, canada_day_month_from)
nz_data4 = SentinelData(coords_nz4, canada_years, canada_day_month_to, canada_day_month_from)


# get wms
#plot_nz_wms1 = nz_data1.get_available_data()
#plot_nz_wms2 = nz_data2.get_available_data()
#plot_nz_wms3 = nz_data3.get_available_data()
#plot_nz_wms4 = nz_data4.get_available_data()
        

    




# Norway. scandy mountains. 
s11, w11 = 61.19427, 6.71053
n11, e11 = 61.39842, 7.14765
  

# BBOX 2 
s12, w12 = 62.11413, 11.91172
n12, e12 = 62.31454, 12.35366


# BBOX 3. nr namsos
s13, w13 = 64.5321, 11.68324
n13, e13 = 64.72623, 12.17067



coords_nor1 = [w11, s11, e11, n11]
coords_nor2 = [w12, s12, e12, n12]
coords_nor3 = [w13, s13, e13, n13]

nor_data1 = SentinelData(coords_nor1, canada_years, canada_day_month_to, canada_day_month_from)
nor_data2 = SentinelData(coords_nor2, canada_years, canada_day_month_to, canada_day_month_from)
nor_data3 = SentinelData(coords_nor3, canada_years, canada_day_month_to, canada_day_month_from)


# get wms
#plot_nor_wms1 = nor_data1.get_available_data()
#plot_nor_wms2 = nor_data2.get_available_data()
#plot_nor_wms3 = nor_data3.get_available_data()

# 5. 


# Argentina. bbox 1. 
s14, w14 = -36.1615, -70.60757
n14, e14 = -36.35363, -70.87261
    

# Argentina BBOX2
s15, w15 = -36.90052, -71.07906
n15, e15 = -36.7103, -70.80853
    
   
 # Argentina BBOX3
s16, w16 = -48.25276, -72.33129
n16, e16 = -48.05762, -72.0168
       
coords_arg1 = [w14, s14, e14, n14]
coords_arg2 = [w15, s15, e15, n15]
coords_arg3 = [w16, s16, e16, n16]

arg_data1 = SentinelData(coords_arg1, canada_years, canada_day_month_to, canada_day_month_from)
arg_data2 = SentinelData(coords_arg2, canada_years, canada_day_month_to, canada_day_month_from)
arg_data3 = SentinelData(coords_arg3, canada_years, canada_day_month_to, canada_day_month_from)


# get wms
#plot_arg_wms1 = arg_data1.get_available_data()
#plot_arg_wms2 = arg_data2.get_available_data()
#plot_arg_wms3 = arg_data3.get_available_data()
  

 
 
# 6. 

 # Alaska bbox1 3 pics
s17, w17 = 60.58666, -144.4895
n17, e17 = 60.78174, -144.04753
    
    
    
# alaska bbox 2 2 pics
s18, w18 = 60.54723, -145.6761
n18, e18 = 60.74312, -145.24079
    

    
# alaska bbox 3. 3 pics
s19, w19 = 60.86118, -146.57959
n19, e19 = 61.05052, -146.14288


# alaska bbox 4. 3 pics 
s20, w20 = 61.10963, -146.62525
n20, e20 = 61.30145, -146.18168


coords_al1 = [w17, s17, e17, n17]
coords_al2 = [w18, s18, e18, n18]
coords_al3 = [w19, s19, e19, n19]
coords_al4 = [w20, s20, e20, n20]


al_data1 = SentinelData(coords_al1, canada_years, canada_day_month_to, canada_day_month_from)
al_data2 = SentinelData(coords_al2, canada_years, canada_day_month_to, canada_day_month_from)
al_data3 = SentinelData(coords_al3, canada_years, canada_day_month_to, canada_day_month_from)
al_data4 = SentinelData(coords_al4, canada_years, canada_day_month_to, canada_day_month_from)


# get wms
#plot_al_wms1 = al_data1.get_available_data()
#plot_al_wms2 = al_data2.get_available_data()
#plot_al_wms3 = al_data3.get_available_data()
#plot_al_wms4 = al_data4.get_available_data()




I 1
outpath CN_bbox2_1.tif
not writing file: CN_bbox2_1.tif
I 2
outpath CN_bbox2_2.tif
not writing file: CN_bbox2_2.tif
I 3
outpath CN_bbox2_3.tif
not writing file: CN_bbox2_3.tif
I 4
outpath CN_bbox2_4.tif
not writing file: CN_bbox2_4.tif
I 5
outpath CN_bbox2_5.tif
not writing file: CN_bbox2_5.tif
I 6
outpath CN_bbox2_6.tif
not writing file: CN_bbox2_6.tif
I 7
outpath CN_bbox2_7.tif
not writing file: CN_bbox2_7.tif
I 8
outpath CN_bbox2_8.tif
not writing file: CN_bbox2_8.tif
I 9
outpath CN_bbox2_9.tif
not writing file: CN_bbox2_9.tif
I 10
outpath CN_bbox2_10.tif
not writing file: CN_bbox2_10.tif
I 11
outpath CN_bbox2_11.tif
not writing file: CN_bbox2_11.tif
I 12
outpath CN_bbox2_12.tif
not writing file: CN_bbox2_12.tif
I 13
outpath CN_bbox2_13.tif
not writing file: CN_bbox2_13.tif
I 14
outpath CN_bbox2_14.tif
not writing file: CN_bbox2_14.tif
I 15
outpath CN_bbox2_15.tif
not writing file: CN_bbox2_15.tif
I 16
outpath CN_bbox2_16.tif
not writing file: CN_bbox2_16.tif
I 17
outpath CN_bbo

KeyboardInterrupt: 

In [20]:
# find the min and max across all files so i can normalise to this
# per folder (green, red, ndfsi, nir, swir)


import imageio
import os
import rasterio as rio
import numpy as np
from osgeo import gdal

path = os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/NDFSI')
files = os.listdir(path)
print("files", files)

min_max = {'band':[1000000,0]}

for fname in files:
    print("fname:", fname)
    ds = gdal.Open(fname)
    array = ds.ReadAsArray()
   # band = ReadAsArray(fname)

    # GET MIN MAX of each img
    mm = (np.amin(array), np.amax(array))
  #  print("nir mm", mm) # print min and max of ALL nir files 
    print('orig nir min[0] is', mm[0])

    #  if observed min is smaller than current min, update with new min
    #  if observed max is bigger than current max, update with new max
    if mm[0] <= min_max['band'][0]: # if the min is less than 0, then make it 0
        min_max['band'][0] = mm[0]
       # print('min max NIR is: ',min_max)
    if mm[1] >= min_max['band'][1]: # if the max is more than 1000000 then make 
        min_max['band'][1] = mm[1]
       # print('min max NIR is: ',min_max)
    # track progress through loop
print(min_max)      

    
    
# MIN/MAX:
# Green: {'band': [0.0, 1.7664]}
# Red: {'band': [0.0, 1.696]}
# NIR: {'band': [0.0, 1.6144]}
# SWIR: {'band': [0.0, 1.5225]}
# NDSI: {'band': [-1.0, 1.0]}
# NDFSI: {'band': [-1.0, 1.0]}


files ['NZ_bbox2_3.tif', 'FR_bbox2_6.tif', 'Arg_bbox1_7.tif', 'Arg_bbox3_5.tif', 'Arg_bbox3_4.tif', 'Nor_bbox3_7.tif', 'Arg_bbox1_5.tif', 'FR_bbox2_4.tif', 'NZ_bbox2_5.tif', 'Arg_bbox1_1.tif', 'NZ_bbox4_3.tif', 'Arg_bbox3_3.tif', 'Arg_bbox3_2.tif', 'Nor_bbox3_3.tif', 'NZ_bbox2_4.tif', 'Arg_bbox1_2.tif', 'FR_bbox2_2.tif', 'FR_bbox1_7.tif', 'CN_bbox2_82.tif', 'Nor_bbox3_14.tif', 'NZ_bbox1_10.tif', 'Nor_bbox3_15.tif', 'Arg_bbox2_7.tif', 'CN_bbox2_83.tif', 'NZ_bbox3_2.tif', 'FR_bbox1_5.tif', 'FR_bbox3_17.tif', 'Arg_bbox2_1.tif', 'CN_bbox2_87.tif', 'FR_bbox1_2.tif', 'Nor_bbox1_14.tif', 'FR_bbox3_15.tif', 'Nor_bbox1_15.tif', 'NZ_bbox1_6.tif', 'Arg_bbox2_2.tif', 'Nor_bbox1_18.tif', 'Al_bbox2_3.tif', 'FR_bbox3_18.tif', 'FR_bbox3_25.tif', 'Al_bbox2_2.tif', 'Al_bbox4_4.tif', 'Al_bbox4_6.tif', 'NZ_bbox1_8.tif', 'FR_bbox1_8.tif', 'NZ_bbox4_13.tif', 'Nor_bbox1_22.tif', 'Al_bbox4_2.tif', 'Al_bbox1_2.tif', 'CN_bbox1_4.tif', 'CN_bbox1_5.tif', 'Al_bbox1_3.tif', 'Al_bbox3_3.tif', 'CN_bbox3_5.tif', 'CN_b

In [93]:
# normalise

# MIN/MAX:
# SWIR: {'band': [0.0, 1.5225]}
# Green: {'band': [0.0, 1.7664]}
# Red: {'band': [0.0, 1.696]}
# NIR: {'band': [0.0, 1.6144]}
# NDSI: {'band': [-1.0, 1.0]}
# NDFSI: {'band': [-1.0, 1.0]}

import cv2
import tifffile as tf
from imageio import imread


#I normalized them to 0-255 float32 and saved as tif. 
#And then I split them and saved my tiles as uint8 tiff.



# os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Green_b03')
#os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Red_b04')
#os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/NDFSI')
os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Downloaded/NDSI')

bands_dir = sorted(os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Downloaded/NDSI'))


for file in bands_dir: 
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Downloaded/NDSI')
   # print("fname:", file)
    
    array = imread(file)

    #print("original pixel values", pixel_value)
    # rescale each pixel between 0-255
    # dividing all pixels values by the largest pixel value to get 0-1 then * 255

    # these two for arrays with minus numbers to start with:
    normalised = array + 1 # bring all -1 values above 0
    normalised = (normalised/2) # because the total range is -1 to 1 so 2
    normalised = normalised * 255
    
    # or for non-minus:
   # normalised = (array/1.6144) * 255 # divide by max, then times to get on 0-255 scale
 
    normalised = normalised.astype('float32')

    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Normalised/Norm_NDSI') 
    out_path = file
    print("outfile", out_path)
    
    imageio.imwrite(out_path, normalised)
    
    #im = Image.fromarray(normalised) #.astype(np.float32)
    # need to use imageio.imwrite(out_path, NDFSI) because image.from array only does rgb
    #im.save(out_path)





outfile Al_bbox1_2.tif
outfile Al_bbox1_3.tif
outfile Al_bbox1_4.tif
outfile Al_bbox2_2.tif
outfile Al_bbox2_3.tif
outfile Al_bbox3_2.tif
outfile Al_bbox3_3.tif
outfile Al_bbox3_4.tif
outfile Al_bbox4_2.tif
outfile Al_bbox4_4.tif
outfile Al_bbox4_6.tif
outfile Arg_bbox1_1.tif
outfile Arg_bbox1_2.tif
outfile Arg_bbox1_5.tif
outfile Arg_bbox1_7.tif
outfile Arg_bbox1_8.tif
outfile Arg_bbox2_1.tif
outfile Arg_bbox2_2.tif
outfile Arg_bbox2_7.tif
outfile Arg_bbox3_2.tif
outfile Arg_bbox3_3.tif
outfile Arg_bbox3_4.tif
outfile Arg_bbox3_5.tif
outfile CN_bbox1_1.tif
outfile CN_bbox1_2.tif
outfile CN_bbox1_4.tif
outfile CN_bbox1_5.tif
outfile CN_bbox2_82.tif
outfile CN_bbox2_83.tif
outfile CN_bbox2_87.tif
outfile CN_bbox3_1.tif
outfile CN_bbox3_2.tif
outfile CN_bbox3_3.tif
outfile CN_bbox3_4.tif
outfile CN_bbox3_5.tif
outfile FR_bbox1_2.tif
outfile FR_bbox1_5.tif
outfile FR_bbox1_7.tif
outfile FR_bbox1_8.tif
outfile FR_bbox2_2.tif
outfile FR_bbox2_4.tif
outfile FR_bbox2_6.tif
outfile FR_bbox3_15

In [106]:
# stack bands
#  red_ndfsi_green !!! DO THIS COMBO
      #  or basic- nir_swir_red
      #  or NSDI_NDFSI_red
    # or NDSI_nir_red
    
        
import os
import glob
import rasterio as rio
import os
import imageio
from skimage.transform import resize



os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Stacked') 

directory_to_check = '/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Stacked/before/red_ndfsi_green'


# Get all the subdirectories of directory_to_check recursively and store them in a list:
directories = [os.path.abspath(x[0]) for x in os.walk(directory_to_check)]
directories.remove(os.path.abspath(directory_to_check)) # If you don't want your main directory included



for subdir in directories:
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Stacked/before/red_ndfsi_green') 
    os.chdir(subdir) 
    list_files = glob.glob('*.tif')
    print("FILE LIST", list_files)
    
    label = list_files[2]
    print("label", label)
    name = label.split(".")[0]
    
    array1 = imread(list_files[0])
    array2 = imread(list_files[1])
    array3 = imread(list_files[2]) 
    
    stack = np.stack((array1, array2, array3), axis =0)
    print("shape", stack.shape)

    # want the three 3 afterwards like width,height,3!
    # the 1,2,0 are indexes for width,height,no.of channels
    stack = stack.transpose((1,2,0))
    
    # set out dir
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Bands/Stacked/after/red_ndfsi_green') 
 
    outfile_tif = name + ".tif" 
    
    # WORKS, BUT .NPY?
    #stack = stack.astype('float32')  # Augmentation package only takes this!?       
   # np.save(new_tif, stack)

    imageio.imwrite(outfile_tif, stack)
        

        
        
        
        
        
        
        
# TIPS ON WRITING MULTIBAND TIFS WITH IMAGEIO:
# can only use imageio for non-rgb pics
# do not use Image package for non-rgb!
# NEED to RESHAPE ARRAY!! i.e. make 3,2000,2000 shape so 3 first
# need to reshape array becuase:
# the transpose operator reverses the index, if the image is stored in RGB x rows x cols the transpose operator will..
# yield cols x rows x RGB (which is the rotated image and not the desired result).
# https://www.py4u.net/discuss/143041  
# https://mxnet.apache.org/versions/1.5.0/tutorials/basic/reshape_transpose.html
    

FILE LIST ['Al_bbox1_2 3.tif', 'Al_bbox1_2 2.tif', 'Al_bbox1_2.tif']
label Al_bbox1_2.tif
shape (3, 2170, 2407)
FILE LIST ['NZ_bbox1_6.tif', 'NZ_bbox1_6 2.tif', 'NZ_bbox1_6 3.tif']
label NZ_bbox1_6 3.tif
shape (3, 2393, 2498)
FILE LIST ['Nor_bbox3_7 2.tif', 'Nor_bbox3_7 3.tif', 'Nor_bbox3_7.tif']
label Nor_bbox3_7.tif
shape (3, 2159, 2323)
FILE LIST ['NZ_bbox4_13 3.tif', 'NZ_bbox4_13 2.tif', 'NZ_bbox4_13.tif']
label NZ_bbox4_13.tif
shape (3, 2424, 2282)
FILE LIST ['FR_bbox1_7.tif', 'FR_bbox1_7 3.tif', 'FR_bbox1_7 2.tif']
label FR_bbox1_7 2.tif
shape (3, 2309, 2462)
FILE LIST ['CN_bbox3_1 3.tif', 'CN_bbox3_1 2.tif', 'CN_bbox3_1.tif']
label CN_bbox3_1.tif
shape (3, 2352, 2450)
FILE LIST ['FR_bbox2_2.tif', 'FR_bbox2_2 2.tif', 'FR_bbox2_2 3.tif']
label FR_bbox2_2 3.tif
shape (3, 2344, 2430)
FILE LIST ['NZ_bbox4_3 2.tif', 'NZ_bbox4_3.tif', 'NZ_bbox4_3 3.tif']
label NZ_bbox4_3 3.tif
shape (3, 2424, 2282)
FILE LIST ['Nor_bbox3_14.tif', 'Nor_bbox3_14 3.tif', 'Nor_bbox3_14 2.tif']
label Nor_bbo

In [ ]:
# splitting for NDSI_NIR_DEM

#And then I split them and saved my tiles as uint8 tiff.

import os
import numpy as np
from PIL import Image
import glob
import os 
from PIL import Image
import sys
import gdal
import image_slicer
import math
from skimage.transform import resize   
import imageio 
import earthpy.plot as ep
import earthpy.spatial as es
import rasterio as rio
import cv2
import tifffile as tf
from imageio import imread


# func to split the images
def split_image(dim_pix, im):
    # Find the number of sub-Images that fit in rows
    rows = []
    tiles = []
    for i in range((math.floor(im.shape[0] / dim_pix))):
        rows.append(i)
    # Find the number of sub-Images that fit in rows
    columns = []
    for i in range((math.floor(im.shape[1] / dim_pix))):
        columns.append(i)

    # Numerically identify the sub-Images
    a = 0
    for i in rows:
        for j in columns:
            # Check for 244 x 244 (Mask) or 244 x 244 x 3 (TC Images)
            if (im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                  0 + dim_pix * i: dim_pix + (dim_pix * i)].shape[0]) == dim_pix:
                if (im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                  0 + dim_pix * i: dim_pix + (dim_pix * i)].shape[1]) == dim_pix:

                    tile = im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                            0 + dim_pix * i: dim_pix + (dim_pix * i)]

                    # Stop white tiles for positive results
                    count = np.count_nonzero(tile == 1) == (dim_pix * dim_pix)
                    if count:
                        all_black = np.tile(1, (dim_pix, dim_pix))
                        tiles.append(tile)
                    else:
                        tiles.append(tile)
                    a += 1
                else:
                    print("Out of shape")
    return tiles



# define tensor dims
input_tensor_dimensions = 244


label_dir = sorted(os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Labels')) # FINAL/ALL_snow_labelled
#print(label_dir)
img_dir = sorted(os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs')) # *.tif Data_collection/FINAL/ALL_stacked_tifs
#print(img_dir)

# label pics:
for i in range(len(label_dir)):
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Labels') # FINAL/ALL_snow_labelled
    # read png labelled pics in as array 
    i_label = label_dir[i]
    name = i_label.split(".")[0]
    #print(i_label)
    array = np.array(Image.open(i_label))
   # print(array)
   # print("arr shape", (np.shape(array)))
    # delete 30 rows and 30 columns from first and last to remove white space before splitting
    np_first_row_del = np.delete(array, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 0)
    np_first_col_del = np.delete(np_first_row_del, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 1)
    
    np_last_row = np.delete(np_first_col_del, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20], 0)
    np_no_whitespace = np.delete(np_last_row, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20], 1)
    
    # resize so white space is adjusted to the new 244x244 dimensions
    array_height = int((np_no_whitespace.shape[0] / 244)) * 244 
    array_width = int((np_no_whitespace.shape[1] / 244)) * 244 
    resized_array = resize(array, (array_height, array_width))
    label_tiles = split_image(dim_pix=input_tensor_dimensions, im=resized_array) # resized_array
    n = len(label_tiles)
    # set out path
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/To_ribble/NDSI_DEM_NIR/PNGLabels') #save path to split labelled pic
    for index in range(n):
        # check indx matches indx of tifs
        #print("label ind", index)
        outfile_png = name + "_" + str(index+1) + ".png"
        im = Image.fromarray((label_tiles[index] * 255).astype(np.uint8))
       # im.save(outfile_png)
    
    # tif pics:
    tif_image = "{}{}".format(name, ".tif") # change to .tif !!!!!!
    
    i_image = "/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs/{}".format(tif_image) # FINAL/ALL_stacked_tifs/
    print("rgb name", i_image)
    #print("rgb image file name",i_image)
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs') # Data_collection/FINAL/ALL_stacked_tifs
    # OpenCV for Python treats image data as ndarray
       
    #tif_array = rio.open(i_image)
    # the -1 is to read in other channels as well
    # or use 1 to read rgb colours
    
    # read tif in as array
   # tif_array = cv2.imread(i_image, -1)

    # read as an array:
    tif_array = imread(tif_image)

    print("orig tif array", tif_array)
    print("tif array", tif_array.shape)
    print("tif shape", tif_array.dtype)
    #print("tif 1 unique", (np.unique(tif_array)))
    
  
    # compare the original pixelxpixel size w the array once rows/cols have been deleted
   # print("tif orig width", tif_array.shape[0])
   # print("tif orig height", tif_array.shape[1])
    #print("dt", (tif_array.dtype))
    #print("tif arr", tif_array)
    # delete whitespace
    np_first_row = np.delete(tif_array, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 0)
    np_first_col = np.delete(np_first_row, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 1)
    np_last_ro = np.delete(np_first_col, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20], 0)
    np_rm_whitespace = np.delete(np_last_ro, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20], 1)

    # resize to fit to new 244x244 with minimal whitespace
    tif_array_height = int((np_rm_whitespace.shape[0] / 244)) * 244 
    print("tif resized array_height", tif_array_height)
    tif_array_width = int((np_rm_whitespace.shape[1] / 244)) * 244
    print("tif resize array_width", tif_array_width)
    
    resized_array = resize(np_rm_whitespace, (tif_array_height, tif_array_width, 3)) 
    #print("shape", np.shape(resized_array))
   # print("tif resized unique", (np.unique(resized_array)))
    print("resized tif array", resized_array)
    
    # split the big pics into small 244 x244 tiles
    basemap_tiles = split_image(dim_pix=input_tensor_dimensions, im=resized_array)
   # print("len basemap tiles", len(basemap_tiles))
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/To_ribble/NDSI_DEM_NIR/float32_jpegs') #JPEGImages save path Data_collection/FINAL/test_tif
    for ind in range(len(label_tiles)): #
        outfile_tif = name + "_" + str(ind+1) + ".tif"
        print("tif ind", ind)
        
        # save as unit8 type for input into model
       # im = Image.fromarray(basemap_tiles[ind]) #* 255).astype(np.unit8)) #* 255 # uint8 #float32_jpegs
        minitile_array = np.array(basemap_tiles[ind])
        # image lib cant handle any other data types but unit8, rgb ones
        imageio.imwrite(outfile_tif, minitile_array) 

# SAVE AS UNIT8!!!    
    
#    im = Image.fromarray(basemap_tiles[ind])
    # im  = np.array((basemap_tiles[ind])).astype('float32')
    #tf.write(outfile_tif, im)

    
